In [27]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
from decimal import Decimal
import pandas as pd
%matplotlib

Using matplotlib backend: Qt5Agg


In [28]:
# density ratio pression spectra analysis
#                                                    name of config file    folder with data
ratios_p1, w_em_p1, results_p1 = functions.get_profile_ratio('../config_p1_2302.json', '../data/2302/2302spectr/')
ratios_p2, w_em_p2, results_p2 = functions.get_profile_ratio('../config_p2_2302.json', '../data/2302/2302spectr/')
n_avg_p1, n_min_p1, n_max_p1  = functions.get_density(ratios_p1, '../fantz-data/fants.txt')
n_avg_p2, n_min_p2, n_max_p2  = functions.get_density(ratios_p2, '../fantz-data/fants.txt')
p1_off = 2.40e-5
p2_off = 1.34e-4

In [29]:
# density ratio with non calibrated intensities
ratios_p1_biased, w_em_p1_biased, results_p1_biased = functions.get_profile_ratio('../config_p1_2302.json', 
                                                             '../data/2302/2302spectr/', unbiased=False)
ratios_p2_biased, w_em_p2_biased, results_p2_biased = functions.get_profile_ratio('../config_p2_2302.json', 
                                                             '../data/2302/2302spectr/', unbiased=False)
p1_off = 2.40e-5
p2_off = 1.34e-4

In [4]:
# create a pandas dataframe for densities with non calibrated data
df_spectra_n_biased = pd.DataFrame()
# for p1
for w, content in results_p1_biased.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p1_off
    df_spectra_n_biased = pd.concat([df_spectra_n_biased, df])
    
# for p2
for w, content in results_p2_biased.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p2_off
    df_spectra_n_biased = pd.concat([df_spectra_n_biased, df])

# apply the get density function to every row and save the results in a density dataframe
df = pd.DataFrame(df_spectra_n_biased['ratio'].apply(functions.get_density).tolist(), columns=['n', 'n_min', 'n_max'])
df_spectra_n_biased = df_spectra_n_biased.reset_index()
del df_spectra_n_biased['index']

# units


# concatenate the two dataframes
df_spectra_n_biased = pd.concat([df_spectra_n_biased, df], axis=1)
# group by power and take average
a = df_spectra_n_biased.groupby(['p_off', 'w', 'id']).mean()
#a

df_spectra_n_biased.columns = ['id', 'p (mbar)', 'ratio', 'w (%)', 'p_off (mbar)',
                       'n ($cm^{-3}$)', 'n_min ($cm^{-3}$)', 'n_max ($cm^{-3}$)']
# print the results of the density with calibration
df_spectra_n_biased

,id,p (mbar),ratio,w (%),p_off (mbar),n ($cm^{-3}$),n_min ($cm^{-3}$),n_max ($cm^{-3}$)
0,a1ad1,0.000000,0.505440,40,0.000024,1.153432e+11,1.031509e+11,1.284972e+11
1,a7ad7,0.000000,0.479926,40,0.000024,1.090003e+11,9.568259e+10,1.216016e+11
2,a3ad3,0.000000,0.472770,40,0.000024,1.070826e+11,9.261592e+10,1.196677e+11
3,a12ad12,0.000039,0.464527,20,0.000024,1.046522e+11,8.908301e+10,1.174397e+11
4,a10ad10,0.000039,0.492524,20,0.000024,1.122959e+11,1.005151e+11,1.250065e+11
5,a14ad14,0.000038,0.460769,20,0.000024,1.034932e+11,8.747235e+10,1.164240e+11
6,a13ad13,0.000039,0.461203,20,0.000024,1.036272e+11,8.765857e+10,1.165414e+11
7,a22ad22,0.000031,0.542278,60,0.000024,1.240351e+11,1.106690e+11,1.384536e+11
8,a21ad21,0.000031,0.540422,60,0.000024,1.235973e+11,1.102903e+11,1.379520e+11
9,a20ad20,0.000031,0.530456,60,0.000024,1.212458e+11,1.082564e+11,1.352585e+11


In [5]:
# create a pandas dataframe for densities
df_spectra_n = pd.DataFrame()
# for p1
for w, content in results_p1.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p1_off
    df_spectra_n = pd.concat([df_spectra_n, df])
    
# for p2
for w, content in results_p2.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p2_off
    df_spectra_n = pd.concat([df_spectra_n, df])

# apply the get density function to every row and save the results in a density dataframe
df = pd.DataFrame(df_spectra_n['ratio'].apply(functions.get_density).tolist(), columns=['n', 'n_min', 'n_max'])
df_spectra_n = df_spectra_n.reset_index()
del df_spectra_n['index']

# units


# concatenate the two dataframes
df_spectra_n = pd.concat([df_spectra_n, df], axis=1)
# group by power and take average
a = df_spectra_n.groupby(['p_off', 'w', 'id']).mean()
#a

df_spectra_n.columns = ['id', 'p (mbar)', 'ratio', 'w (%)', 'p_off (mbar)',
                       'n ($cm^{-3}$)', 'n_min ($cm^{-3}$)', 'n_max ($cm^{-3}$)']
# print the results of the density with calibration
df_spectra_n

NameError: name 'results_p1' is not defined

In [6]:
# evaluate the difference between biased and unbiased data
biased = df_spectra_n.groupby(['p_off (mbar)', 'w (%)']).mean()
unbiased = df_spectra_n_biased.groupby(['p_off (mbar)', 'w (%)']).mean()
index = unbiased.index
ratio_biased = biased['ratio'].values
ratio_unbiased = unbiased['ratio'].values
n_biased = biased['n ($cm^{-3}$)'].values
n_unbiased = unbiased['n ($cm^{-3}$)'].values
df = pd.DataFrame([ratio_biased, ratio_unbiased, n_biased, n_unbiased]).T
df.columns =  ['ratio bias', 'ratio unbias', 'biased', 'unbiased']
df.index = index
df

KeyError: 'p_off (mbar)'

In [7]:
# create pandas dataframe for temperatures
# open the json file
df_spectra_T = pd.DataFrame()
with open('../config_p1_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p1_off
    df_spectra_T = pd.concat([df_spectra_T, df])

with open('../config_p2_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p2_off
    df_spectra_T = pd.concat([df_spectra_T, df])
    
df_spectra_T = df_spectra_T.reset_index()
df_spectra_T = df_spectra_T.rename(columns={'index': 'id'})

folder_path = '../data/2302/2302spectr/'
extension = '.dat'

df_spectra_T['file'] = folder_path + df_spectra_T['id'] + extension
df_spectra_T['wavelength'] = 750.4
df_spectra_T['i_abs'] = df_spectra_T.apply(lambda x: functions.get_absolute_intensity_2(x['file'],x['t'], x['wavelength']), axis=1)
df_spectra_T.columns = ['id', 'p (mbar)', 't (s)', 'w (%)', 'p_off (mbar)', 'file',
                       'wavelength (nm)', 'i_abs ($cm^{-3} s^{-1}$)']

# time to shoot myself
df = pd.merge(df_spectra_T, df_spectra_n, on=['w (%)', 'p_off (mbar)'])
#df['p_correction'] = 1/1.29
a = df.groupby(['p_off (mbar)', 'w (%)'], as_index=False).mean()
a['pec ($cm^{3}s^{-1}$)'] = a.apply(lambda x: functions.get_pec(x['i_abs ($cm^{-3} s^{-1}$)'], x['p_off (mbar)'], x['n ($cm^{-3}$)']), axis=1)
a

KeyError: 'w (%)'

In [8]:
# adas reading and interpolation
df_adas = functions.read_adas_file('../data/adas/ar_750.dat')
mesh_n, mesh_pec = np.meshgrid(np.log10(a['n ($cm^{-3}$)']), np.log10(a['pec ($cm^{3}s^{-1}$)']))
a['T'] = functions.interpolate_dataframe(df_adas, (np.log10(a['n ($cm^{-3}$)']), np.log10(a['pec ($cm^{3}s^{-1}$)'])))
df_spectr = a
df_spectr = df_spectr.groupby(['p_off (mbar)', 'w (%)']).mean()
df_spectr

KeyError: 'n ($cm^{-3}$)'

In [9]:
# langmuir density and profile temperature
#                        path to results data

# reading fit results
df1 = agg.aggregate_data('../langmuir/data/data-2302')
df2 = agg.aggregate_data('../langmuir/data/data-2302-Te')
df_tot = pd.merge(df1, df2, left_index=True, right_index=True, left_on='W_em', right_on='id', how = 'inner')
#raggruppo
df_data = df_tot.groupby(['P_off_x', 'W_em_y'])['n_e_x', 'err_n_e_x', 'T_e_y', 'err_T_e_y'].mean()
df_data

Reading dataset:['2302lang10', '2302lang07', '2302lang08', '2302lang09', '2302lang06', '2302lang11', '2302lang03', '2302lang02', '2302lang01', '2302lang05', '2302lang04', '2302lang12']
Reading dataset:['2302lang10', '2302lang07', '2302lang08', '2302lang09', '2302lang06', '2302lang11', '2302lang03', '2302lang02', '2302lang01', '2302lang05', '2302lang04', '2302lang12']


n_e_x     err_n_e_x     T_e_y  err_T_e_y
P_off_x  W_em_y                                                 
0.000024 20      1.111022e+17  1.503052e+12  4.571680   0.264776
         40      2.074349e+17  1.005729e+12  4.166896   0.304953
         60      2.984036e+17  4.423558e+12  4.386303   0.269850
0.000134 20      2.125159e+17  3.654460e+12  3.952657   0.091311
         40      2.713786e+17  1.273970e+12  4.381683   0.112396
         60      3.090086e+17  5.339409e+12  4.197886   0.184088

In [10]:
# plotting langmuir and spectra data
pressures = df_data.index.get_level_values(0).unique()
i = 0
j = 2
plt.suptitle('Argon', fontsize=15)
for pressure in pressures:
    w = df_data.xs(pressure).index.values
    plt.subplot(221 + i)
    plt.title('Pressure: {:.2e} mbar'.format(pressure))
    T_e_lang = df_data.xs(pressure)['T_e_y']
    T_e_spectr = df_spectr.xs(pressure)['T']
    err_T_e= df_data.xs(pressure)['err_T_e_y']
    plt.errorbar(w, T_e_lang, yerr=err_T_e, label='Langmuir', fmt='o')
    plt.errorbar(w, T_e_spectr, yerr=err_T_e, label='OES', fmt='o')
    plt.xlabel('W (%)', fontsize=13)
    plt.ylabel('$T_e$ (eV)', fontsize=13)
    plt.grid()
    plt.legend()
    
    plt.subplot(221 + j)
    n_e_lang = df_data.xs(pressure)['n_e_x'] / 1e6
    n_e_spectr = df_spectr.xs(pressure)['n ($cm^{-3}$)']
    err_n_e_spectr = df_spectr.xs(pressure)['n_max ($cm^{-3}$)'] - df_spectr.xs(pressure)['n_min ($cm^{-3}$)']
    err_n_e_lang = df_data.xs(pressure)['err_n_e_x'] / 1e6
    plt.errorbar(w, n_e_lang, yerr=err_n_e_lang, label='Langmuir', fmt='o')
    plt.errorbar(w, n_e_spectr, yerr=err_n_e_spectr, label='OES', fmt='o')    
    plt.xlabel('W (%)', fontsize=13)
    plt.ylabel('$n_e (cm^{-3})$', fontsize=13)
    plt.grid()
    plt.legend()
    plt.tight_layout()
    i = i + 1
    j = j + 1 

NameError: name 'df_spectr' is not defined